In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%run imports.py

In [35]:
import dim_models

In [36]:
shape = [10, 10, 300]
X = torch.randn(10, *shape)
lin1 = dim_models.DimLinear(shape, -1, out_features=301)
lin2 = nn.Linear(300, 301)
lin3 = dim_models.DimLinearManual(300, 301, shape=shape, dim_to_mix=2)

print(util.count_params(lin1))
print(util.count_params(lin2))
print(util.count_params(lin3))

90601
90601
90601


In [37]:
%%time
a = lin1(X)

CPU times: user 16.8 ms, sys: 204 µs, total: 17 ms
Wall time: 16.1 ms


In [38]:
%%time
a = lin2(X)

CPU times: user 1.93 ms, sys: 2.07 ms, total: 4.01 ms
Wall time: 3.31 ms


In [39]:
%%time
a = lin3(X)

CPU times: user 79.3 ms, sys: 69.3 ms, total: 149 ms
Wall time: 148 ms


In [58]:
net1 = nn.LayerNorm([10, 20, 30], )
net2 = dim_models.DimLayerNorm([10, 20, 30], dims_to_mix=(0, 1, 2))

X = torch.randn(1000, 10, 20, 30)

print(util.count_params(net1))
print(util.count_params(net2))

12000
12000


In [59]:
print([p.shape for p in net1.parameters()])
print([p.shape for p in net2.parameters()])

[torch.Size([10, 20, 30]), torch.Size([10, 20, 30])]
[torch.Size([10, 20, 30]), torch.Size([10, 20, 30])]


In [66]:
%%time
net1(X).shape

CPU times: user 8.08 ms, sys: 9.58 ms, total: 17.7 ms
Wall time: 16.3 ms


torch.Size([1000, 10, 20, 30])

In [67]:
%%time
net2(X).shape

CPU times: user 60.6 ms, sys: 5.96 ms, total: 66.6 ms
Wall time: 65.7 ms


torch.Size([1000, 10, 20, 30])

In [49]:
X = torch.randn(100, 200, 300, 4)

In [59]:
%%timeit
a = X.mean(dim=0).std(dim=0).mean(dim=0).std(dim=0)

2.65 ms ± 92.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [62]:
%%timeit
a = X.reshape(300, 4, 100, 200).mean(dim=0).std(dim=0).mean(dim=0).std(dim=0)

5.21 ms ± 41.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [63]:
%%timeit
a = X.permute(3, 2, 1, 0).mean(dim=0).std(dim=0).mean(dim=0).std(dim=0)

42.2 ms ± 1.24 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
import dim_models

In [126]:
[p.shape for p in d3.parameters()]

[torch.Size([100, 99, 1, 1, 1]), torch.Size([100])]

In [92]:
M = torch.randn(99, 100)
d0 = nn.Linear(99, 100)
d1 = nn.Conv1d(99, 100, 1)
d2 = nn.Conv2d(99, 100, 1)
d3 = nn.Conv3d(99, 100, 1)

In [113]:
X1 = torch.randn(50, 60, 70, 99, 1)
X2 = torch.randn(50, 60, 99, 1, 70)
X3 = torch.randn(50, 99, 1, 60, 70)
X4 = torch.randn(99, 1, 50, 60, 70)

In [114]:
%%time
(M.T@X1).shape

CPU times: user 106 ms, sys: 71.6 ms, total: 177 ms
Wall time: 19.5 ms


torch.Size([50, 60, 70, 100, 1])

In [115]:
%%time
d0(X1[..., 0]).shape

CPU times: user 173 ms, sys: 66.1 ms, total: 239 ms
Wall time: 25.9 ms


torch.Size([50, 60, 70, 100])

In [116]:
%%time
(M*X1).sum(dim=-2, keepdim=True).shape

CPU times: user 2.98 s, sys: 3.45 s, total: 6.43 s
Wall time: 789 ms


torch.Size([50, 60, 70, 1, 100])

In [117]:
%%time
(M[:, :, None]*X2).sum(dim=-3, keepdim=True).shape

CPU times: user 3.69 s, sys: 2.86 s, total: 6.56 s
Wall time: 804 ms


torch.Size([50, 60, 1, 100, 70])

In [118]:
%%time
d1(X2[..., 0, :].reshape(-1, 99, 70)).reshape(50, 60, 100, 70).shape

CPU times: user 204 ms, sys: 177 ms, total: 381 ms
Wall time: 39 ms


torch.Size([50, 60, 100, 70])

In [119]:
%%time
d2(X3[..., 0, :, :].reshape(-1, 99, 60, 70)).reshape(50, 100, 60, 70).shape

CPU times: user 218 ms, sys: 183 ms, total: 400 ms
Wall time: 42.7 ms


torch.Size([50, 100, 60, 70])

In [120]:
%%time
d3(X4[..., 0, :, :, :].reshape(-1, 99, 50, 60, 70)).reshape(100, 50, 60, 70).shape

CPU times: user 108 ms, sys: 102 ms, total: 210 ms
Wall time: 24 ms


torch.Size([100, 50, 60, 70])

In [77]:
x = torch.randn(10, 10, 10, 10, 50)

In [72]:
conv = nn.Conv1d(99, 100, 1)

In [75]:
conv(torch.randn(1,77, 99, 30)).shape

RuntimeError: Expected 3-dimensional input for 3-dimensional weight [100, 99, 1], but got 4-dimensional input of size [1, 77, 99, 30] instead

In [40]:
num_patches = (224//14)**2
patch_dim_ori = 14*14*3
patch_dim = 1280
num_patches, patch_dim

(256, 1280)

In [45]:
(1280*5120*2+256*640*2)*32

429916160

In [47]:
(1280*2)*32

81920

In [65]:
class DimLayerNorm(nn.LayerNorm):
    def __init__(self, shape):
        super().__init__(shape)
        
#         for p in self.parameters():
#             p.data = torch.ones(10, 20, 1)*p.data.mean().item()
        print([p.shape for p in self.parameters()])
    def forward(self, x):
        return super().forward(x)
    

In [66]:
net = DimLayerNorm([10, 20, 30])
[p.shape for p in net.parameters()]

[torch.Size([10, 20, 30]), torch.Size([10, 20, 30])]


[torch.Size([10, 20, 30]), torch.Size([10, 20, 30])]

In [67]:
net(torch.randn(77, 10, 20, 30)).shape

torch.Size([77, 10, 20, 30])

In [141]:
class LayerNorm(nn.LayerNorm):
    def __init__(self, shape):
        super().__init__(shape)
        
    def forward(self, x):
        u = x.mean(dim=(-1), keepdim=True)
        std = (x.var(dim=(-1), keepdim=True, unbiased=False) + 1e-5).sqrt()
        
        x = (x-u)/std
        
        x = x*self.weight+self.bias
        
        return x
    
net1 = nn.LayerNorm([1000])
net2 = LayerNorm([1000])

x = (torch.randn(7777, 1000))

(net1(x)-net2(x)).abs().max()

tensor(9.5367e-07, grad_fn=<MaxBackward1>)

In [146]:
%%time
a = net1(x)

CPU times: user 3.68 ms, sys: 142 µs, total: 3.82 ms
Wall time: 3.06 ms


In [147]:
%%time
a = net2(x)

CPU times: user 245 ms, sys: 2.64 ms, total: 248 ms
Wall time: 25.9 ms
